In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import urllib

def engine():
    connection_string = (
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=localhost,1433;"
        "DATABASE=Olympic_Data;"
        "UID=sa;"
        "PWD=Admin@123;"
        "Encrypt=yes;"
        "TrustServerCertificate=yes;"
    )

    params = urllib.parse.quote_plus(connection_string)

    return create_engine(f"mssql+pyodbc:///?odbc_connect={params}", fast_executemany=True)

def read_query(file_path):
    with open(file_path, 'r') as file:
        query = file.read()
    return query

def load_athletes(engine):
    athlete_query = '../queries/athletes_query.sql'
    athletes = read_query(athlete_query)
    return pd.read_sql(athletes, engine)

def load_results(engine):
    results_query = '../queries/results_query.sql'
    results = read_query(results_query)
    return pd.read_sql(results, engine) 

db_engine = engine()

athletes = load_athletes(db_engine)
results = load_results(db_engine)

print("Athletes Data:")
display(athletes.head())  

print("Results Data:")
display(results.head())

/Users/jonathanli/vscode/Olympic_Athletes/env/lib/python3.14/site-packages/pandas/io/sql.py:1648: SAWarning: Unrecognized server version info '17.0.4015.4'.  Some SQL Server features may not function properly.
  con = self.exit_stack.enter_context(con.connect())


Athletes Data:


,Roles,Sex,Used Name,Born,Died,NOC,athlete_id,Measurements
0,Competed in Olympic Games,Male,Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1,None
1,Competed in Olympic Games,Male,Arnaud•Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",None,France,2,183 cm / 76 kg
2,Competed in Olympic Games • Administrator,Male,Jean•Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...","17 July 1994 in Arbonne, Pyrénées-Atlantiques ...",France,3,183 cm / 76 kg
3,Competed in Olympic Games,Male,Jacques•Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)","20 March 1978 in Monaco, Monaco (MON)",France,4,168 cm / 64 kg
4,Competed in Olympic Games,Male,Albert•Canet,"17 April 1878 in Wandsworth, England (GBR)","25 July 1930 in Paris VIIe, Paris (FRA)",France,5,None


Results Data:


,Games,Event,Team,Pos,Medal,As,athlete_id,NOC,Discipline,Nationality,Unnamed: 7
0,1912 Summer Olympics,"Single Sculls, Men (Olympic)",None,2 h3 r1/4,None,Axel Matias Haglund,36767,FIN,Rowing,None,NaN
1,1952 Summer Olympics,"Eights, Men (Olympic)",Finland,3 h2 r2/5,None,Yrjö Hakoila,36768,FIN,Rowing,None,NaN
2,1976 Summer Olympics,"Coxless Pairs, Men (Olympic)",Leo Ahonen,8.0,None,Kari Hanska,36769,FIN,Rowing,None,NaN
3,1912 Summer Olympics,"Coxed Fours, Outriggers, Men (Olympic)",Helsingfors R. K.,2 h1 r2/4,None,Valdemar Henriksson,36770,FIN,Rowing,None,NaN
4,1992 Summer Olympics,"Double Sculls, Men (Olympic)",Reima Karppinen,13.0,None,Esko Hillebrandt,36771,FIN,Rowing,None,NaN


## Cleaning Tasks

- Get rid of bullet points in Used Names
- Split height/weight
- Parse out dates from Born & Died column
- Parse out city, region, and country from Born column
- Get rid of extra column

Get rid of dot

In [ ]:
df_athletes = athletes.copy()

count = df_athletes['Used name'].str.contains('•', na=False).sum()
print(count)

df_athletes['Name'] = df_athletes['Used name'].str.replace('•', ' ')

145115


Use a small data set to clean before applying to athlete dataset

In [3]:
small_data = pd.DataFrame(
    ['180 cm', '183 cm / 67 kg', '/ 65 cm', '68 kg /'], columns=['Measurements']
)
small_data['height_cm'] = small_data['Measurements'].str.extract(r'(\d+)\s*cm')
small_data['weight_kg'] = small_data['Measurements'].str.extract(r'(\d+)\s*kg')


In [23]:
small_data['height_cm'] = pd.to_numeric(small_data['height_cm'])
small_data['weight_kg'] = pd.to_numeric(small_data['weight_kg'])

Split height & weight into differnet columns

In [4]:
df_athletes['height_cm'] = df_athletes['Measurements'].str.extract(r'(\d+)\s*cm')
df_athletes['weight_kg'] = df_athletes['Measurements'].str.extract(r'(\d+)\s*kg')

In [5]:
df_athletes['height_cm'] = pd.to_numeric(df_athletes['height_cm'])
df_athletes['weight_kg'] = pd.to_numeric(df_athletes['weight_kg'])

Seperating date from location

In [6]:
date_pattern = r'(\d+ \w+ \d{4})'
df_athletes['born_date'] = df_athletes['Born'].str.extract(date_pattern)
df_athletes['death_date'] = df_athletes['Died'].str.extract(date_pattern)

Check if all rows follow this pattern
- keep as NaN if it doesnt follow

In [18]:
df_athletes[~df_athletes['Born'].str.match(date_pattern, na=False)].head(5)

,Roles,Sex,Full name,Used name,Born,Died,NOC,athlete_id,Measurements,Affiliations,...,height_cm,weight_kg,born_date,death_date,Born_year,Death_year,Birth_location,City,Region,Country
12,Competed in Olympic Games,Male,J.•Defert,J.•Defert,None,None,France,13,None,"Racing Club de France, Paris (FRA)",...,<NA>,<NA>,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
13,Competed in Olympic Games,Male,Étienne•Durand,Étienne•Durand,None,None,France,14,None,"TCP, Paris (FRA)",...,<NA>,<NA>,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
27,Competed in Olympic Games • Competed in Olympi...,Male,Guy•Lejeune,"Guy, Baron•Lejeune",None,None,France,28,None,None,...,<NA>,<NA>,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
28,Competed in Olympic Games,Male,Albert•Lippmann,Albert•Lippmann,None,None,France,29,None,"unattached, (MIX)",...,<NA>,<NA>,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
91,Competed in Olympic Games,Male,Lionel Hunter•Escombe,Lionel•Escombe,"1876 in Natal, KwaZulu-Natal (RSA)","15 October 1914 in Brentford, England (GBR)",Great Britain,92,None,None,...,<NA>,<NA>,NaT,1914-10-15,1876,1914,"Natal, KwaZulu-Natal (RSA)",<NA>,<NA>,<NA>


Add a year column because now all rows follow the same date pattern

In [8]:
df_athletes['Born_year'] = df_athletes['Born'].str.extract(r'(\d{4})')
df_athletes['Death_year'] = df_athletes['Died'].str.extract(r'(\d{4})')

Change born & death date into timedate objects

In [9]:
date_columns = ['born_date', 'death_date']
df_athletes[date_columns] = df_athletes[date_columns].apply(pd.to_datetime, errors='coerce')

Parse out dates from 'born' and 'died' columns

In [10]:
# working with a small data set first
birth_data = pd.DataFrame([
    '1876 in Natal, '
    'KwaZulu-Natal (RSA)', 
    '12 December 1886 in Bordeaux, Gironde (FRA)', 
    'in Niort, Deux-Sèvres (FRA)'], 
    columns=['Born']
)

location_pattern = r'in (.*)'
birth_data['birth_loc'] = birth_data['Born'].str.extract(location_pattern)


In [13]:
birth_data

,Born,birth_loc
0,"1876 in Natal, KwaZulu-Natal (RSA)","Natal, KwaZulu-Natal (RSA)"
1,"12 December 1886 in Bordeaux, Gironde (FRA)","Bordeaux, Gironde (FRA)"
2,"in Niort, Deux-Sèvres (FRA)","Niort, Deux-Sèvres (FRA)"


Now apply to the athletes dataframe

In [11]:
location_pattern = r'in (.*)'
df_athletes['Birth_location'] = df_athletes['Born'].str.extract(location_pattern)

In [12]:
seperation_pattern = r'([\w\s]+), ([\w\s]+) \((\w+)\)'
df_athletes[['City', 'Region', 'Country']] = df_athletes['Birth_location'].str.extract(seperation_pattern)

Convert columns into its logical datatype

In [14]:
athlete_cols_int = ['athlete_id', 'Born_year', 'Death_year', 'height_cm', 'weight_kg']
df_athletes[athlete_cols_int] = df_athletes[athlete_cols_int].apply(pd.to_numeric, errors='coerce').astype('Int64')

athlete_cols_str = ['Name', 'Birth_location', 'City', 'Region', 'Country', 'NOC']
df_athletes[athlete_cols_str] = df_athletes[athlete_cols_str].astype("string")

In [15]:
columns_keep = ['athlete_id', 'Name', 'Sex', 'NOC', 'height_cm', 'weight_kg', 'born_date', 'death_date', 'Born_year', 'Death_year', 'City', 'Region', 'Country']

df_athletes_clean = df_athletes[columns_keep]

In [17]:
df_athletes_clean.head(5)

,athlete_id,Name,Sex,NOC,height_cm,weight_kg,born_date,death_date,Born_year,Death_year,City,Region,Country
0,1,Jean-François Blanchy,Male,France,<NA>,<NA>,1886-12-12,1960-10-02,1886,1960,Bordeaux,Gironde,FRA
1,2,Arnaud Boetsch,Male,France,183,76,1969-04-01,NaT,1969,<NA>,Meulan,Yvelines,FRA
2,3,Jean Borotra,Male,France,183,76,1898-08-13,1994-07-17,1898,1994,<NA>,<NA>,<NA>
3,4,Jacques Brugnon,Male,France,168,64,1895-05-11,1978-03-20,1895,1978,Paris VIIIe,Paris,FRA
4,5,Albert Canet,Male,France,<NA>,<NA>,1878-04-17,1930-07-25,1878,1930,Wandsworth,England,GBR


In [19]:
df_athletes_clean.to_csv('../data/processed/athletes.csv', index=False)

### Clean Results csv

- Remove = and DNS into NA
- Add a seasons and year column
- Parse Male events from Female events

In [ ]:
df_results = results.copy()


In [20]:
# Replcae the = with ' '
df_results['Pos_clean'] = df_results['Pos'].str.replace('=', ' ')

# Convert into a numeric object to turn all DNF and DNS into NAN
df_results['Pos_clean'] = pd.to_numeric(df_results['Pos_clean'], errors='coerce')

Find games that dont match the pattern
- One game spans 2 years
- Some games dont have a season

In [6]:
games_pattern = r'(\d{4} \w+ \w+)'
df_results[~df_results['Games'].str.contains(games_pattern, na=False)]

/var/folders/zd/qbk_l4lx5_v5pwtr7z4cryt40000gn/T/ipykernel_362/2399546709.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_results[~df_results['Games'].str.contains(games_pattern, na=False)]


,Games,Event,Team,Pos,Medal,As,athlete_id,NOC,Discipline,Nationality,Unnamed: 7,Pos_clean
121776,1888-89 Zappas Olympic Games,"Rope Climbing, Men ()",None,1,None,Sotirios Versis,55911,GRE,Artistic Gymnastics (Gymnastics),None,None,1.0


In [21]:
# Use the start year of the event
year = r'(\d{4})'
df_results['Games_Year'] = df_results['Games'].str.extract(year)

# Extract the season then set NA to the games without a season
season_pattern = r'\d{4} (\w+)'
df_results['Season'] = df_results['Games'].str.extract(season_pattern)

season_list = ['Winter', 'Summer', 'Fall', 'Spring']
df_results.loc[~df_results['Season'].isin(season_list), 'Season'] = pd.NA

Gender

In [22]:
# Filter out males and females
genders = r'\b(Men|Women)\b'
df_results['Gender'] = df_results['Event'].str.extract(genders)


In [23]:
discipline_pattern = r'\s\(.*\)'
df_results['Discipline_clean'] = df_results['Discipline'].str.replace(discipline_pattern, ' ', regex=True)

In [24]:
df_results['Name'] = df_results['As'].str.replace('-', ' ')

In [ ]:
event_pattern = r'(.*), '
df_results['Event_clean'] = df_results['Event'].str.extract(event_pattern)

Convert medals into integers

In [26]:
# remove whitespace and change into lowercase to normalize
df_results['Medal'] = df_results['Medal'].str.strip().str.lower()

medal_map = {
    'gold': 3,
    'silver': 2,
    'bronze': 1,
}

df_results['Points'] = df_results['Medal'].map(medal_map).fillna(0)

Medalist or non-medlaist

In [51]:
df_results['Preformance_Result'] = df_results['Points'].apply(lambda x: 'Medalist' if x > 0 else 'non-Medalist')


Remove redundant columns & assign appropriate datatype

In [ ]:
results_columns_keep = ['athlete_id', 'Name', 'Gender', 'Discipline_clean', 'Event_clean', 'Medal', 'Points', 'Preformance_Result', 'Pos_clean', 'Games_Year', 'Season']
df_results = df_results[results_columns_keep]

df_results.tail(5)

,athlete_id,Name,Gender,Discipline_clean,Event_clean,Medal,Points,Preformance_Result,Pos_clean,Games_Year,Season
308403,148983,Marián Skupek,Men,Luge,Singles,None,0.0,non-Medalist,26.0,2022,Winter
308404,148984,Elsa Fermbäck,Women,Alpine Skiing,Slalom,None,0.0,non-Medalist,28.0,2022,Winter
308405,148985,Hilma Lövblom,NaN,Alpine Skiing,Team,None,0.0,non-Medalist,13.0,2022,Winter
308406,148985,Hilma Lövblom,Women,Alpine Skiing,Giant Slalom,None,0.0,non-Medalist,NaN,2022,Winter
308407,148986,Charlotta Säfvenberg,Women,None,Slalom,None,0.0,non-Medalist,24.0,2022,Winter


In [55]:
results_cols_str = ['Name', 'Gender', 'Discipline_clean', 'Event_clean', 'Medal', 'Season', 'Preformance_Result']
results_cols_int = ['athlete_id', 'Points', 'Pos_clean', 'Games_Year']

df_results_clean = df_results[results_cols_str + results_cols_int].copy()

df_results_clean[results_cols_str] = df_results_clean[results_cols_str].astype('string')
df_results_clean[results_cols_int] = df_results_clean[results_cols_int].apply(pd.to_numeric, errors='coerce').astype('Int64')

df_results_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308408 entries, 0 to 308407
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Name                308408 non-null  string
 1   Gender              284954 non-null  string
 2   Discipline_clean    308407 non-null  string
 3   Event_clean         306583 non-null  string
 4   Medal               44139 non-null   string
 5   Season              305807 non-null  string
 6   Preformance_Result  308408 non-null  string
 7   athlete_id          308408 non-null  Int64 
 8   Points              308408 non-null  Int64 
 9   Pos_clean           234554 non-null  Int64 
 10  Games_Year          308408 non-null  Int64 
dtypes: Int64(4), string(7)
memory usage: 27.1 MB


In [56]:
df_results_clean.to_csv('../data/processed/results.csv')

Group into age groups
- Merge df_athletes and df_results together

In [ ]:

columns = ['athlete_id','height_cm', 'weight_kg', 'Born_year', 'Death_year', 'Country']

df_merge = df_results_clean.merge(df_athletes_clean[columns], on='athlete_id', how='left')



Find the age groups of athletes

In [58]:
df_merge['Age'] = df_merge['Games_Year'] - df_merge['Born_year']
df_merge['Age'] = pd.to_numeric(df_merge['Age'], errors='coerce')

# Youngest age 11 and oldest 73
df_merge['Age'].min()
df_merge['Age'].max()

# Any Na
df_merge[df_merge['Age'].isna()]

# Group them accordingly
age_bin = [13, 20, 30, 40, 50, 60, 70, 80]

age_groups = np.array(['11-12', '13-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79'])

index = np.digitize(df_merge['Age'].fillna(-1), age_bin)

df_merge['Age_group'] = age_groups[index]

# For the NA ages change the gorup to unknown
df_merge.loc[df_merge['Age'].isna(), 'Age_group'] = 'Unknown'




In [59]:
df_merge[df_merge['Age_group'] == 'Unknown'].head(5)

,Name,Gender,Discipline_clean,Event_clean,Medal,Season,Preformance_Result,athlete_id,Points,Pos_clean,Games_Year,height_cm,weight_kg,Born_year,Death_year,Country,Age,Age_group
50,J. Defert,Men,Tennis,Singles,<NA>,Summer,non-Medalist,13,0,8,1896,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Unknown
51,J. Defert,Men,Tennis,Doubles,<NA>,Summer,non-Medalist,13,0,<NA>,1896,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Unknown
52,Étienne Durand,Men,Tennis,Singles,<NA>,Summer,non-Medalist,14,0,8,1900,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Unknown
53,Étienne Durand,Men,Tennis,Doubles,<NA>,Summer,non-Medalist,14,0,5,1900,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Unknown
54,Étienne Durand,Men,Tennis,"Singles, Handicap",<NA>,Summer,non-Medalist,14,0,9,1900,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Unknown


Calculate BMI

In [60]:
df_merge['BMI'] = (df_merge['weight_kg'] / (df_merge['height_cm'] / 100)**2)

Find the points for each age group for each Olympic year
- Store it in a seperate df because if we have 1000 athletes in a certain age group for a year it will repeate it 1000 times

In [61]:
# Store in seperate df
df_year_total_points = df_merge.groupby(['Games_Year', 'Age_group'])['Points'].sum().reset_index()

df_year_total_points.to_csv('../data/processed/year_total_points.csv')

Find podium appearances percentage of the age groups and their disicipline
- Because the age group 20-29 has more participants they will always have more points so here calculate podium appearance percentage

In [62]:
# Count by medal since points has values of 0
df_podium_appearance_age = df_merge.groupby(['Games_Year', 'Age_group', 'Discipline_clean']).agg({
    'athlete_id' : 'count',
    'Medal' : 'count'
}).reset_index()

df_podium_appearance_age['Appearance_%'] = ((df_podium_appearance_age['Medal'] / df_podium_appearance_age['athlete_id']) * 100).round(2)

# Rename for clarity
df_podium_appearance_age = df_podium_appearance_age.rename(columns={
    'Games_Year' : 'Year',
    'Age_group' : 'Age_Group',
    'athlete_id' : 'Total_Athletes',
    'Medal' :'Podium_Appearances',
})
df_podium_appearance_age
df_podium_appearance_age.to_csv('../data/processed/podium_appearances_age.csv')

The mean & std of medalist vs non-medalist in their discipline

In [63]:
df_physical_preformance = df_merge.groupby(['Games_Year', 'Discipline_clean', 'Preformance_Result']).agg({
    'height_cm': ['mean', 'std'],
    'weight_kg': ['mean', 'std'],
}).reset_index()

df_physical_preformance.to_csv('../data/processed/physical_preformance.csv')
df_physical_preformance.head()


Games_Year      Discipline_clean Preformance_Result   height_cm             \
                                                             mean        std   
0       1888  Artistic Gymnastics        non-Medalist        <NA>       <NA>   
1       1896  Artistic Gymnastics            Medalist        <NA>       <NA>   
2       1896  Artistic Gymnastics        non-Medalist       160.0        0.0   
3       1896             Athletics           Medalist  175.173913  10.806783   
4       1896             Athletics       non-Medalist      168.52   9.698649   

   weight_kg             
        mean        std  
0       <NA>       <NA>  
1       62.0        0.0  
2       62.0        0.0  
3       73.6  11.979808  
4  70.163265  16.794527